### Segmenting and Clustering Neighborhoods in Toronto
This Notebook will do:
- Scraping data from wikipedia page using BeautifulSoup package
- Processing data


In [163]:
import numpy as np
import pandas as pd
import urllib3
from bs4 import BeautifulSoup

In [164]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#print (r.data)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [165]:
#soup = BeautifulSoup(r.data, 'html.parser')
#print(soup.prettify())

In [166]:
#f = open("wiki_toronto.html", "w")
#f.write(soup.prettify())
#print ("COMPLETED: Save web html with name, wiki_toronto.html")
#f.close()

In [167]:
import re

bs = BeautifulSoup(r.data, 'html.parser')
table_body=bs.find('tbody')
#print (table_body)

#declare some list as global
neighbor_list = []
columns = []
nb_data = []

tr_rows = bs.find_all('tr')
#print (tr_rows)

In [168]:
initial=True
#print (len(tr_rows))

for tr_row in tr_rows:
    #print (tr_row)
    #print ("-----------------")
    if initial==True:
        col=[]
        th=tr_row.find_all('th')
        #print (th)
        for th_row in th:
            #print (th_row.text)
            col.append(str(th_row.text).rstrip())
        
        initial=False
        
    else:
        data=[]
        td=tr_row.find_all('td')
        #print (td)
        for td_row in td:
            #print (td_row.text)
            data.append(str(td_row.text).rstrip())
            
        #append data into list
        nb_data.append(data)
    
        print ("*************")
        print ("col=",col)
        print ("data=",data)
        print ("*************")
    
columns = col.copy()

print ("###########################")
print ("columns = ",columns)
print ("nb_data = ",nb_data)
print ("###########################")
    
print ("Completed for web scraping!")

*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M1A', 'Not assigned', 'Not assigned']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M2A', 'Not assigned', 'Not assigned']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M3A', 'North York', 'Parkwoods']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M4A', 'North York', 'Victoria Village']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M5A', 'Downtown Toronto', 'Harbourfront']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M5A', 'Downtown Toronto', 'Regent Park']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M6A', 'North York', 'Lawrence Heights']
*************
*************
col= ['Postcode', 'Borough', 'Neighbourhood']
data= ['M6A', 'North York', 'Lawrence Manor']
*************
*************
col= ['Postcode', 'B

In [169]:
#columns = col.copy()
#print (columns)
df_toronto = pd.DataFrame(columns=columns)
df_toronto.head()

len_eq_3 = [(len(item) == 3) for item in nb_data]
#len_eq_3

toronto_neiborhoods_list=[]
for i, cond in zip(range(0,len(len_eq_3)),len_eq_3):
    #print (i,cond)
    if cond==True:
        toronto_neiborhoods_list.append(nb_data[i])

#print ("Processed data: toronto_neiborhoods_list = ",toronto_neiborhoods_list)

#toronto_neighborhoods = nb_data[1:-5]
#df for 
df_toronto = pd.DataFrame(data=toronto_neiborhoods_list, columns=columns)

df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [170]:
# only get the Borough having value, ignore 'Not assigned'
df_toronto_1 = df_toronto[(df_toronto['Borough']!='Not assigned')].reset_index(drop=True)
df_toronto_1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [171]:
#Neighbourhood have 'Not assigned' value replace the same name of Borough
for i,item in zip(range(0,len(df_toronto_1['Neighbourhood'])),df_toronto_1['Neighbourhood']):
    #print (i,item)
    if item=='Not assigned':
        df_toronto_1.loc[i,'Neighbourhood'] = df_toronto_1.loc[i,'Borough']
        
df_toronto_1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


- reset index

In [172]:
df_toronto_1.reset_index(drop=True, inplace=True)
df_toronto_1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods **separated with a comma as shown in row 11** in the above table.

In [173]:
#df_toronto_1['Neighbourhood'].astype(str)
#df_toronto_1.dtypes

df_toronto_1.groupby(['Postcode','Borough'],sort=False).agg(lambda x: ','.join(x))

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


- use the **.shape** method to print the number of rows of your dataframe.

In [174]:
df_toronto_1.shape

(211, 3)

**COMPLETE DATA SCRAPING AND PROCESSING!!!**